In [1]:
import pandas as pd
import geopandas as gpd
import os

In [2]:
os.chdir(os.getcwd()+"/Data")

In [3]:
#read in data, reorganize and concatenate
scores1112 = pd.read_csv('1112_ACT.csv')
scores1213 = pd.read_csv('1213_ACT.csv')
scores1314 = pd.read_csv('1314_ACT.csv')
scores1415 = pd.read_csv('1415_ACT.csv')
scores1516 = pd.read_csv('1516_ACT.csv')
scores1617 = pd.read_csv('1617_ACT.csv')

scores1112['year'] = 2011
scores1213['year'] = 2012
scores1314['year'] = 2013
scores1415['year'] = 2014
scores1516['year'] = 2015
scores1617['year'] = 2016

col_list = ['RCDTS','District Name/ School Name',' Enrollment ','ACT Score','year']

scores1112 = scores1112[col_list]
scores1213 = scores1213[col_list]
scores1314 = scores1314[col_list]
scores1415 = scores1415[col_list]
scores1516 = scores1516[col_list]
scores1617 = scores1617[col_list]

df_list = [scores1112,scores1213,scores1314,scores1415,scores1516,scores1617]

all_scores = pd.concat(df_list)
all_scores = all_scores[all_scores['ACT Score'].isnull()==False]

all_scores.reset_index(inplace=True)

In [4]:
#import geojson
schools_geo = gpd.read_file('Chicago Public Schools - School Locations SY1718.geojson')

In [6]:
schools_geo.head()

,network,grade_cat,zip,governance,school_id,ald_15,long,grades,commarea,short_name,ward_15,address,geo_network,lat,phone,geometry
0,Charter,ES,60632,Charter,400009,Edward M. Burke,-87.7400974358,"K, 1, 2, 3, 4, 5, 6, 7, 8",GARFIELD RIDGE,GLOBAL CITIZENSHIP,14,4647 W 47TH ST,8,41.8075785077,1(773)582-1100,POINT (-87.74009743581296 41.80757850688568)
1,Charter,HS,60609,Charter,400010,Patricia R. Dowell,-87.6258490365,"9, 10, 11, 12",WASHINGTON PARK,ACE TECH HS,3,5410 S STATE ST,9,41.7961215104,1(773)548-8705,POINT (-87.62584903655835 41.79612150956602)
2,Charter,ES,60612,Charter,400011,Jason C. Ervin,-87.7052345259,"K, 1, 2, 3, 4, 5, 6, 7, 8",EAST GARFIELD PARK,LOCKE A,28,3141 W JACKSON BLVD,5,41.877248353,1(773)265-7232,POINT (-87.70523452593643 41.87724835219521)
3,Charter,HS,60618,Charter,400013,Ariel E. Reboyras,-87.7270956585,"9, 10, 11, 12",AVONDALE,ASPIRA - EARLY COLLEGE HS,30,3986 W BARRY AVE,4,41.9372982853,1(773)252-0970 x137,POINT (-87.72709565849537 41.93729828444937)
4,Charter,ES,60625,Charter,400017,Rey Colon,-87.7218246652,"6, 7, 8",ALBANY PARK,ASPIRA - HAUGAN,35,3729 W LELAND AVE,1,41.966405668,1(773)252-0970,POINT (-87.72182466520648 41.96640566718369)


In [5]:
school_progress11 = pd.read_csv(
    'Chicago_Public_Schools_-_Progress_Report_Cards__2011-2012_.csv')

In [32]:
#parse phone number for joining
def fix_string(row):
    new_row = row.replace("(","").replace(")","").replace("-","").replace(" ","")
    return new_row
                
school_progress11['parsed_num'] = school_progress11['Phone Number'] \
                                                .apply(lambda row: fix_string(row))

In [33]:
school_progress11 = school_progress11[['School ID','parsed_num','Name of School','Latitude','Longitude']]

In [34]:
rcdts_file = pd.read_csv('rcdts.csv')

In [35]:
rcdts_file['RCDTS'] = rcdts_file['Region'].astype(str) \
                        + rcdts_file['Type'].astype(str) \
                            + rcdts_file['School'].astype(str)

In [36]:
#parse phone number for joining - rcdts file 
def fix_other_string(row):
    new_row = str(row)
    new_row = new_row.replace("-","").replace(" ","")
    return new_row
                
rcdts_file['parsed_Num'] = rcdts_file['Telephone'] \
                                                .apply(lambda row: fix_other_string(row))

In [37]:
#join rcdts 
rcdts_joined = pd.merge(left=rcdts_file,right=school_progress11,
                             left_on='parsed_Num',right_on='parsed_num')

In [38]:
rcdts_joined = rcdts_joined[['School ID','RCDTS','Name of School','Latitude','Longitude']]

In [39]:
#join rcdts to score data
score_joined = pd.merge(left=all_scores,right=rcdts_joined,how='left',left_on='RCDTS',right_on='RCDTS')

In [40]:
score_joined = score_joined[['School ID','RCDTS','ACT Score','District Name/ School Name',' Enrollment ','year']]

In [41]:
score_joined.to_csv('score_joined_dirty.csv')

In [42]:
score_joined = score_joined[score_joined['School ID'].isnull()==False]

In [43]:
schools_geo['school_id'] = schools_geo['school_id'].astype(float)

In [44]:
score_joined = pd.merge(left=score_joined,right=schools_geo,how='left',
                                    left_on='School ID',right_on='school_id')

score_joined = score_joined[['School ID','RCDTS','ACT Score','District Name/ School Name',
              ' Enrollment ','year','commarea','geometry']]

In [45]:
score_joined.to_csv('score_data_cleaned.csv')